# Multilayer Perceptron

Trong bài này, sinh viên được hướng dẫn sử dụng Tensorflow để hiện thực Multilayer Perceptron (neural network).

Code được tham khảo từ một số nguồn sau đây:
* [Aymeric Damien (aymericdamien)](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py)

Dataset: the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/). Links: [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).

In [36]:
from __future__ import print_function

import tensorflow as tf

In [38]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

# load dataset
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [39]:
# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [40]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

**Cách đầu tiên, sử dụng các phép toán đơn giản để định nghĩa MLP**

In [41]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

** Định nghĩa MLP gọn hơn với các phép toán đã được tensorflow hỗ trợ sẵn, ở đây ta dùng *tf.layers.dense* **

*Sinh viên chỉ dùng một trong 2 version của hàm multilayer_perceptron, ở đây hoặc ở phía trên*

In [42]:
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, units=n_hidden_1, kernel_initializer=tf.contrib.layers.xavier_initializer())
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, units=n_hidden_2, kernel_initializer=tf.contrib.layers.xavier_initializer())
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, units=n_classes, kernel_initializer=tf.contrib.layers.xavier_initializer())
    return out_layer

Ở đây, thay vì khởi tạo các matrix weights và biases với random_normal, ta dùng xavier_initializer, nó có ảnh hưởng nhất định tới tốc độ hội tụ của model

Ở đây, chúng ta bắt đầu cấu trúc graph, định nghĩa loss (cost) và optimizer để đưa vào training

In [ ]:
# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

Train model và test để xem kết quả

In [ ]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            print(c)
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

2.34943
2.04857
1.67747
1.43097
1.31243
1.1327
0.96959
0.742686
0.693254
0.686071
0.600853
0.683438
0.584435
0.607473
0.681777
0.55044
0.542628
0.526256
0.610948
0.436861
0.508297
0.595685
0.478119
0.347922
0.512103
0.641174
0.605536
0.541008
0.491601
0.308072
0.481602
0.569012
0.745813
0.278165
0.43734
0.511086
0.375967
0.426132
0.494983
0.292147
0.417848
0.488251
0.673391
0.580111
0.34321
0.297439
0.556248
0.284445
0.471229
0.458489
0.382222
0.458367
0.300141
0.480945
0.397956
0.299496
0.465734
0.603727
0.397809
0.353394
0.285771
0.420397
0.39497
0.403572
0.333257
0.494103
0.441461
0.582436
0.291521
0.235694
0.329923
0.450314
0.540157
0.606917
0.309386
0.300417
0.448919
0.417635
0.518384
0.561483
0.324976
0.281087
0.494899
0.31019
0.36086
0.326027
0.41937
0.263508
0.442345
0.328752
0.297703
0.609757
0.264785
0.527874
0.377711
0.313844
0.481662
0.322646
0.273522
0.384724
0.29321
0.310669
0.400622
0.557186
0.269309
0.310882
0.296689
0.283009
0.400546
0.437016
0.242597
0.39004
0.20612
0

0.298011
0.306132
0.129286
0.458168
0.254974
0.325617
0.210361
0.361256
0.214902
0.180847
0.335107
0.531358
0.270003
0.261402
0.27605
0.340889
0.395361
0.316993
0.275166
0.379276
0.345273
0.526731
0.32824
0.214462
0.18313
0.531132
0.287234
0.356512
0.24514
0.606017
0.13523
0.405252
0.379525
0.324834
0.266259
0.344441
0.456801
0.256199
0.537153
0.337759
0.140875
0.173429
0.310864
0.392685
0.250072
0.368701
0.163507
0.305199
0.710831
0.306291
0.18035
0.151388
0.400427
0.303717
0.259044
0.288786
0.3429
0.154102
0.411513
0.226233
0.410079
0.339076
0.273429
0.259554
0.248067
0.359086
0.194143
0.0945691
0.243889
0.219069
0.428372
0.44551
0.335859
0.424049
0.268472
0.367747
0.180386
0.291993
0.438706
0.462697
0.25304
0.301709
0.145611
0.357619
0.290002
0.21257
0.309428
0.139606
0.263934
0.312153
0.208216
0.622256
0.255119
0.205005
0.264551
0.145804
0.248482
0.259145
0.57958
0.228013
0.362136
0.102675
0.177101
0.281736
0.445098
0.315439
0.212186
0.26656
0.371919
0.399596
0.54745
0.233789
0.394

0.213118
0.240939
0.32852
0.222521
0.193522
0.22733
0.205373
0.196054
0.334
0.281385
0.414444
0.47524
0.260131
0.187015
0.262514
0.372926
0.177085
0.376309
0.343669
0.295039
0.298105
0.260393
0.240809
0.420013
0.29664
0.344913
0.19245
0.385632
0.160517
0.337851
0.333048
0.424791
0.125391
0.2248
0.321387
0.369335
0.216891
0.260094
0.303768
0.186089
0.229888
0.344044
0.164883
0.324921
0.267492
0.234407
0.235691
0.431128
0.111949
0.209557
0.546608
0.159902
0.482236
0.426982
0.239313
0.393257
0.385946
0.235138
0.31079
0.173919
0.177583
0.419731
0.10746
0.177154
0.185416
0.391563
0.177604
0.170557
0.327719
0.24835
0.350826
0.487666
0.32321
0.217401
0.563548
0.436415
0.30223
0.545844
0.310421
0.416937
0.249511
0.431817
0.143158
0.39217
0.353762
0.24503
0.324324
0.205577
0.378553
0.157126
0.365007
0.372083
0.33142
0.290994
0.455403
0.163325
0.264993
0.196616
0.295555
0.347211
0.295206
0.19488
0.307166
0.312793
0.247943
0.500009
0.45277
0.292194
0.436494
0.397036
0.190147
0.310834
0.209498
0.4

0.299131
0.158794
0.231757
0.233131
0.261598
0.531908
0.248001
0.125421
0.217099
0.566744
0.291163
0.339744
0.253629
0.318529
0.520281
0.183586
0.349823
0.282034
0.294521
0.263832
0.195082
0.353609
0.340401
0.278947
0.327845
0.26721
0.214534
0.30893
0.292892
0.214966
0.209281
0.394486
0.206001
0.194047
0.135895
0.381546
0.282478
0.172535
0.344655
0.350071
0.151361
0.51125
0.336929
0.319408
0.360517
0.206685
0.191886
0.270519
0.223968
0.423205
0.227216
0.3152
0.3159
0.403555
0.245947
0.23044
0.416175
0.574861
0.302488
0.2346
0.134227
0.332759
0.3715
0.179495
0.233343
0.239327
0.220001
0.269755
0.174068
0.277583
0.210909
0.477241
0.225971
0.413903
0.427669
0.344254
0.290111
0.292291
0.13794
0.142985
0.24528
0.273844
0.496081
0.291667
0.392693
0.306066
0.264488
0.294106
0.250156
0.260747
0.249024
0.266479
0.182927
0.336427
0.309658
0.237284
0.403457
0.443706
0.276806
0.165674
0.231396
0.238422
0.248324
0.401159
0.20557
0.274191
0.416281
0.167991
0.330067
0.267752
0.477905
0.24988
0.212135

0.177728
0.17882
0.313665
0.301735
0.248889
0.166318
0.291023
0.204625
0.298721
0.160127
0.399581
0.391962
0.29317
0.230444
0.160725
0.147168
0.220285
0.380119
0.282271
0.272528
0.195842
0.125299
0.426895
0.302587
0.13635
0.378865
0.219843
0.315718
0.18224
0.308264
0.261707
0.24551
0.426399
0.21167
0.211221
0.3307
0.298614
0.129144
0.311912
0.18369
0.415123
0.485061
0.374026
0.201342
0.238606
0.145861
0.300571
0.239372
0.303369
0.257579
0.275321
0.297932
0.267339
0.329248
0.317725
0.305398
0.292867
0.147903
0.420837
0.187884
0.190704
0.389066
0.167608
0.110736
0.452835
0.13604
0.30292
0.652913
0.581765
0.205109
0.287008
0.396771
0.309921
0.16138
0.10307
0.285633
0.304486
0.267926
0.278983
0.296895
0.229601
0.341423
0.379343
0.175261
0.232049
0.201378
0.450128
0.286315
0.247037
0.41718
0.168435
0.275833
0.307129
0.399796
0.366102
0.466008
0.274748
0.246076
0.231091
0.215615
0.355733
0.249435
0.426326
0.315006
0.194838
0.381058
0.163922
0.261239
0.211904
0.15704
0.385437
0.24543
0.479549

0.327539
0.256733
0.208293
0.35271
0.146048
0.374646
0.129801
0.29828
0.253483
0.250912
0.381807
0.263533
0.353973
0.266682
0.405061
0.223243
0.235198
0.320589
0.319756
0.264366
0.336949
0.179657
0.225121
0.302371
0.279813
0.297432
0.244961
0.0944113
0.30313
0.308797
0.34295
0.223481
0.184543
0.276534
0.418537
0.259253
0.325072
0.390968
0.227741
0.151969
0.368223
0.360878
0.499511
0.154672
0.171902
0.235014
0.206444
0.175515
0.206913
0.16155
0.507193
0.144586
0.257602
0.204045
0.210665
0.221534
0.178067
0.236065
0.208657
0.237258
0.298056
0.327963
0.293293
0.112262
0.159379
0.165107
0.133684
0.490426
0.150944
0.549201
0.187113
0.304966
0.243036
0.282258
0.304957
0.227857
0.129846
0.11508
0.264405
0.216391
0.330988
0.280496
0.274639
0.235812
0.240982
0.295386
0.209652
0.231939
0.514179
0.378178
0.386618
0.567656
0.403881
0.195652
0.232569
0.430941
0.394221
0.375258
0.303355
0.355819
0.433696
0.172723
0.297229
0.368831
0.502326
0.253667
0.250203
0.180302
0.391717
0.510175
0.197937
0.2220

## Bài tập

Bài nộp của sinh viên là chính là **file này** sau khi được đổi tên thành **MSSV.E10_Multilayer_Perceptron.ipynb** và đừng quên ghi thông tin sinh viên vào các ô ở dưới.

Địa chỉ nộp bài: https://www.dropbox.com/request/cmGOid3eqfC87NDza002

Deadline nộp bài: **10:00 thứ 2 tuần tiếp theo**

*Điểm bài này sẽ được tổng hợp với điểm chấm trên lớp (nếu có) để ra điểm cuối cùng, do đó sinh viên vắng học trên lớp dù làm bài có toàn vẹn cũng có thể nhận được điểm thấp hơn điểm tối đa*

Thông tin sinh viên:

In [ ]:
sid = 'MSSV'
name = 'Tên Bạn'

import tensorflow as tf
import numpy as np

# sinh viên import các thư viện cần thiết ở đây

### Bài 1
Trong ví dụ ở trên, chúng ta đã sử dụng toàn bộ tập dữ liệu để train. 

Sinh viên hãy chia tập dữ liệu train ra làm 2 phần riêng biệt ~90% để train và ~10% còn lại dùng để validation. (sửa trực tiếp ở code ví dụ ở trên)

Hãy cho biết kích thước của mỗi tập sau khi tách ra.

Hãy add code validation sau mỗi K steps.

Hãy cho biết các nhận xét của kết quả khi chia ra 2 tập rời như vậy

### Bài 2
Hãy sử dụng tập dữ liệu [cifar 10](https://www.cs.toronto.edu/~kriz/cifar.html)

Hãy xử lý dữ liệu, thay đổi model (nếu cần) và điều chỉnh các thông số cần thiết phù hợp với dữ liệu này.

Sinh viên hãy train, test và cho biết kết quả đạt được (yêu cầu giữ kết quả output khi chạy train, test lúc nộp bài ở các cell tiếp theo ở dưới

In [ ]:
# data load, pre-processing

In [ ]:
# model (if need)

In [ ]:
# train, test (keep output when submit your exercise)
with tf.Session() as sess:
    # train, test code here
    pass

### Bài 3
Các bạn đã train để tạo ra được model sử dụng trong classification đơn giản, tuy nhiên chúng ta chưa lưu lại các kết quả đạt được để tái sử dụng, do đó, mỗi lần test lại sẽ cần train lại mô hình, tốn kém thời gian.

Hãy tìm hiểu và hiện thực (vào code mẫu ở trên) để lưu lại model và tái sử dụng lại sau này.